<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Closed_Water_Air.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [1]:
from dotmap import DotMap
import pandas as pd
import jax
from jax.lax import stop_gradient
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision
from tools.tree_array_transform import VSC, Comp, Range
import tools.che as che
R=8.314 # J/(mol K)

In [2]:
p = che.Props(['Nitrogen','Oxygen', 'Argon', 'Water'])

In [5]:
def model(c,r):
    # c: combination of adjustable variables and static state parameters
    # r: DotMap - store intermediate results for reporting
    r.Pw = p.Pvap(c.T)[3]
    r.V_vap = c.V_tot - c.Vw_i # Approximation - water in the vapor phase is negligible
    W_n_vap = r.Pw * r.V_vap / (R * c.T)
    P = (c.air_n + W_n_vap) * R * c.T / r.V_vap
    # VSC is intended for solving models with multiple equations and expects
    # an iterable of (left,right) tuples
    return ( (c.P_f, P), )

c=DotMap()
c.W_tot = 1. # 1 kg
c.V_tot = 0.01 # 10 Liters
c.P_i = 1e5 # Pa air pressure
c.P_f = 2e5 # Pa final pressure
c.T_i = 298.

c.Vw_i = c.W_tot/p.rhol(c.T_i)[3]
c.Vair_i = c.V_tot - c.Vw_i
c.air_n = c.P_i * c.Vair_i / (R * c.T_i)

c.T = Range(360,300,600)

In [6]:
# transform our model into one that takes a single array of adjustable variables
# for minimization.  Unnecessary for this simple model with just 1 variable, 
# but very helpful for more complex problems later.
vsc=VSC(c,model)
vsc.solve()

0.001973574785335787
0.0006606517658790154
0.0006606517658790154
1.5272366214779895e-05
1.5272366214779895e-05
1.5272366214779895e-05
1.213645372822225e-08
1.213645372822225e-08
1.213645372822225e-08
6.02015987314522e-14
6.02015987314522e-14
6.02015987314522e-14
6.02015987314522e-14
6.02015987314522e-14
6.409037068602645e-20


In [7]:
# Dataframe of adjustable variables
vsc.vdf

,vector1
,1
T,365.719235


In [8]:
# Dataframe of intermediate results
vsc.rdf

,vector1
,1
Pw,77275.424480
V_vap,0.008995


In [9]:
# Dataframe of static state parameters
vsc.sdf

,vector1
,1
V_tot,0.010000
P_i,100000.000000
Vair_i,0.008995
P_f,200000.000000
W_tot,1.000000
T_i,298.000000
air_n,0.363057
Vw_i,0.001005


In [13]:
def model2(c,r):
    Pw = p.Pvap(c.T)[3]
    r.V_vap = c.V_tot - c.Vw_i 
    r.W_n_vap = Pw * r.V_vap / (R * c.T) 
    P = (c.air_n + r.W_n_vap) * R * c.T / r.V_vap

    # tuples from left and right side of equations
    P_constraint = (c.P_f, P)
    W_n_constraint = (r.W_n_vap, c.W_n_vap_desired)
    return (P_constraint, W_n_constraint)

In [14]:
c=DotMap()
c.W_tot = 1. # 1 kg
c.V_tot = 0.01 # 10 Liters
c.P_i = 1e5 # Pa air pressure
c.P_f = 2e5 # Pa final pressure
c.T_i = 298.
c.W_n_vap_desired = 0.3

c.Vw_i = c.W_tot/p.rhol(c.T_i)[3]
c.Vair_i = c.V_tot - c.Vw_i
c.air_n = c.P_i * c.Vair_i / (R * c.T_i)

c.T = Range(350, 300, 400)
c.V_tot = Range(0.015, 0., 0.03)

vsc=VSC(c,model2)
vsc.solve()

0.10764694306583505
0.030440692345800494
0.030440692345800494
0.0012712658793009844
0.0012712658793009844
1.553728074595461e-05
1.553728074595461e-05
1.553728074595461e-05
2.9016926539896234e-09
2.9016926539896234e-09
2.9016926539896234e-09
3.105225229281413e-14
3.105225229281413e-14
3.105225229281413e-14
3.105225229281413e-14
3.105225229281413e-14
8.924222365022703e-20


In [15]:
vsc.vdf

,vector1
,1
T,370.005638
V_tot,0.011204


In [16]:
vsc.rdf


,vector1
,1
V_vap,0.010199
W_n_vap,0.300000


In [21]:
def model3(c, r):
    Pw = p.Pvap(c.T)[3]
    # volume of liquid water lost as water vapor ignored in model1
    # evaluate the density of water at T, but do not propagate gradients through.
    r.v_w_vap = c.W_n_vap*p.Mw[3]/1000/p.rhol(stop_gradient(c.T))[3]
    V_vap = c.V_tot - (c.Vw_i - r.v_w_vap)
    r.W_n_vap = Pw * V_vap / (R * c.T)
    P = (c.air_n + r.W_n_vap) * R * c.T / V_vap
    P_constraint = (c.P_f, P)
    W_constraint = (c.W_n_vap, r.W_n_vap)
    return (P_constraint,W_constraint)

In [22]:
c=DotMap()
c.W_tot = 1. # 1 kg
c.V_tot = 0.01 # 10 Liters
c.P_i = 1e5 # Pa air pressure
c.P_f = 2e5 # Pa final pressure
c.T_i = 298.
c.Vw_i = c.W_tot/p.rhol(c.T_i)[3]
c.Vair_i = c.V_tot - c.Vw_i
c.air_n = c.P_i * c.Vair_i / (R * c.T_i)
c.T = Range(350., 300., 400.)
c.W_n_vap = Range(1., 0., 2.) # moles of water vapor

vsc=VSC(c,model3)
vsc.solve(verbosity=1)

0.6079035669169227
0.38267727987803424
0.12068145433502979
0.12068145433502979
0.09566348468144975
0.09566348468144975
0.0954432158444961
0.0954432158444961
0.0954432158444961
0.0954432158444961
0.0954432158444961
0.0954432158444961
0.0954432158444961
0.09488835758089913
0.09034001876457592
0.09034001876457592
0.09034001876457592
0.077859932311968
0.077859932311968
0.06998075159133216
0.06998075159133216
0.025061259616434288
0.006596757073373889
0.0025185389699037175
9.762275165273431e-05
4.4128965794412945e-05
4.038416547024041e-08
4.038416547024041e-08
2.4609258425528205e-08
2.669105787540159e-13
2.669105787540159e-13
2.669105787540159e-13
2.669105787540159e-13
1.0558646880910466e-17
1.0558646880910466e-17
1.0558646880910466e-17
1.0558646880910466e-17
1.0558646880910466e-17
2.7211155009691965e-23


In [23]:
vsc.rdf

,vector1
,1
v_w_vap,0.000004
W_n_vap,0.228854


In [24]:
vsc.rdf.values # raw numbers
# only 4e-6 m3 of liquid water is lost to the vapor phase compared to the initial
# 1e-3 m3 of liquid water.

array([[4.22513566e-06],
       [2.28853665e-01]])

In [25]:
vsc.vdf

,vector1
,1
T,365.737128
W_n_vap,0.228854
